# Imports and Setups

In [1]:
! pip install transformers==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 21.5 MB/s 
     |████████████████████████████████| 880 kB 20.7 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 86 kB 3.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=729bb1b1b83640012352f6bddc42388610177cfa284d200d1cbf7836dc4599f9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install nlpaug==1.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 405 kB 5.3 MB/s 


In [3]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [4]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=9e8df348e6b529c571cad658aafc4b4e751c12974b375ad850e8deb73d7af274
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [5]:
! git clone https://github.com/nng555/ssmba

Cloning into 'ssmba'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 74 (delta 29), reused 60 (delta 15), pack-reused 0
Unpacking objects: 100% (74/74), done.


Lendo o Arquivo

In [6]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [7]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [78]:
num_gen = 5
perc = 100

In [79]:
arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_trec_'+str(perc)+'_original.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [80]:
base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

In [81]:
import re
import json
# from sklearn.model_selection import train_test_split



def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json['texto']:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "\n"
    f.write(data)




build_text_files(base_cru_1p,'train_dataset.txt')

In [82]:
def build_label_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json['indices']:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "\n"
    f.write(data)




build_label_files(base_cru_1p,'label_dataset.txt')

Só rodar quando gerar o original

In [83]:
! python ssmba/ssmba.py \
  --model bert-base-uncased \
  --in-file  train_dataset.txt \
  --label-file label_dataset.txt \
  --output-prefix ssmba_out \
  --noise-prob 0.25 --num-samples 5 \
  --topk 10

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [84]:
prafraseador = pd.read_csv('ssmba_out', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['parafraseador'])
label = pd.read_csv('ssmba_out.label', encoding='utf-8', sep='//', decimal = '\t', header=None , names = ['indices'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [85]:
fim = pd.concat([prafraseador, label], axis=1)
fim.head()

,parafraseador,indices
0,what in the oldest age?,7
1,what is the teaching profession ।,7
2,what remains the oldest profession?,7
3,what is the teaching profession?,7
4,what is the oldest profession?,7


In [86]:
export = fim.merge(base_cru_1p, how = 'left', on = 'indices')
export.head()

,parafraseador,indices,texto,label
0,what in the oldest age?,7,What is the oldest profession ?,3
1,what is the teaching profession ।,7,What is the oldest profession ?,3
2,what remains the oldest profession?,7,What is the oldest profession ?,3
3,what is the teaching profession?,7,What is the oldest profession ?,3
4,what is the oldest profession?,7,What is the oldest profession ?,3


In [87]:
export = export.drop('indices', axis= 1)
export.head()

,parafraseador,texto,label
0,what in the oldest age?,What is the oldest profession ?,3
1,what is the teaching profession ।,What is the oldest profession ?,3
2,what remains the oldest profession?,What is the oldest profession ?,3
3,what is the teaching profession?,What is the oldest profession ?,3
4,what is the oldest profession?,What is the oldest profession ?,3


In [88]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_trec_'+str(perc)+'_ssmba'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

In [89]:
1000/(2*50+6)

9.433962264150944